# 📝 Task 1: Live Text Update vs. Button-Triggered Update
    
# Requirement
    Build a simple Dash app that includes:
    - An input box (dcc.Input) where the user types text.
    - Two ways to update the text display:
      1. Live Update: The displayed text updates dynamically as the user types.
      2. Button-Triggered Update: The displayed text updates only when a button is clicked.
    
# Example Layout
    - dcc.Input: User enters text.
    - html.Button: Triggers the text update (for State example).
    - html.H3: Displays the entered text.
    

In [6]:
pip install dash

Note: you may need to restart the kernel to use updated packages.


In [1]:
import dash
from dash import dcc, html, Input, Output, State

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Input(id='input-live', type='text', placeholder='Type here...', debounce=False),
    html.H3(id='live-output', children='Live update text will appear here'),
    
    dcc.Input(id='input-button', type='text', placeholder='Type here for button update...'),
    html.Button('Update Text', id='update-button', n_clicks=0),
    html.H3(id='button-output', children='Button update text will appear here')
])

@app.callback(
    Output('live-output', 'children'),
    Input('input-live', 'value')
)
def update_live_text(value):
    return value if value else "Live update text will appear here"

@app.callback(
    Output('button-output', 'children'),
    Input('update-button', 'n_clicks'),
    State('input-button', 'value')
)
def update_button_text(n_clicks, value):
    return value if n_clicks > 0 and value else "Button update text will appear here"

if __name__ == '__main__':
    app.run_server(debug=True   , port=8053)


# 📝 Task 2: Interactive Chart with Multiple Inputs

# Requirement

Build a data visualization dashboard where:
- A user selects a country from a dropdown menu.
- A slider allows adjusting the year range for data filtering.
- A line chart (`dcc.Graph`) updates dynamically to show trends based on both inputs.

# Example Layout
- dcc.Dropdown: Select a country.
- dcc.RangeSlider: Select a year range.
- dcc.Graph: Displays filtered data as a line chart.


In [2]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd
import numpy as np


df = pd.DataFrame({
    'Year': np.tile(np.arange(2000, 2023), 5),
    'Country': np.repeat(['USA', 'Canada', 'Germany', 'India', 'Japan'], 23),
    'Value': np.random.randint(50, 500, 115)
})


app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H2("Interactive Chart with Multiple Inputs"),
    
    html.Label("Select a Country:"),
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': c, 'value': c} for c in df['Country'].unique()],
        value='USA',
        clearable=False
    ),
    
    html.Label("Select Year Range:"),
    dcc.RangeSlider(
        id='year-slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=[2005, 2015],
        marks={year: str(year) for year in range(df['Year'].min(), df['Year'].max()+1, 2)},
        step=1
    ),
    
    dcc.Graph(id='line-chart')
])


@app.callback(
    Output('line-chart', 'figure'),
    Input('country-dropdown', 'value'),
    Input('year-slider', 'value')
)
def update_chart(selected_country, selected_years):
    filtered_df = df[(df['Country'] == selected_country) &
                     (df['Year'] >= selected_years[0]) &
                     (df['Year'] <= selected_years[1])]
    
    fig = px.line(filtered_df, x='Year', y='Value', title=f'Trends in {selected_country}', markers=True)
    return fig

if __name__ == '__main__':
    app.run_server(debug=True   , port=8054)

# 📝 Task 3: Multiple Outputs - Interactive UI Updates

# Requirement
Create an interactive dashboard where clicking a button updates two components simultaneously:
- A text component (`html.H3`) displaying how many times the button has been clicked.
- A background color of a `html.Div`, which changes color randomly on each click.

# Example Layout
- html.Button: Click to trigger updates.
- html.H3: Displays the number of clicks.
- html.Div: Background color changes dynamically.

In [3]:
import dash
from dash import dcc, html, Input, Output
import random


def random_color():
    return f'rgb({random.randint(0,255)}, {random.randint(0,255)}, {random.randint(0,255)})'


app = dash.Dash(__name__)


app.layout = html.Div([
    html.Button('Click Me', id='click-button', n_clicks=0),
    html.H3(id='click-count', children='Button clicked 0 times'),
    html.Div(id='color-box', style={'width': '200px', 'height': '100px', 'backgroundColor': 'lightgrey', 'marginTop': '20px'})
])


@app.callback(
    [Output('click-count', 'children'),
     Output('color-box', 'style')],
    Input('click-button', 'n_clicks')
)
def update_output(n_clicks):
    new_color = random_color()
    return f'Button clicked {n_clicks} times', {'width': '200px', 'height': '100px', 'backgroundColor': new_color, 'marginTop': '20px'}


if __name__ == '__main__':
    app.run_server(debug=True  , port=8055)

# 📌 Task 4: Interactive Scatter Plot with User Controls using NumPy & Dash

## 🔹 Objective
Build an interactive scatter plot where users can dynamically select:

- The **X-axis feature** (e.g., numerical values from the dataset).
- The **Y-axis feature** (another numerical column).

We will generate synthetic data using NumPy instead of using built-in datasets like Iris.

## 🔹 How It Works

### 1️⃣ Generate Random Data using NumPy
Create a dataset with **500 random points**:

```python
num_samples = 500

# Feature_A: Normally distributed values around 50 with a standard deviation of 15 (X)
np.random.normal(50, 15, num_samples)

# Feature_B: Random values uniformly distributed between 10 and 100 (Y)
np.random.uniform(10, 100, num_samples)

# Feature_C: Random integer values between 1 and 100 (Z)
np.random.randint(1, 100, num_samples)
```

### 2️⃣ Build an Interactive Dashboard using Dash
- Users can choose **X and Y** or **Z and Y** features from dropdown menus.
- A **scatter plot** dynamically updates based on the selections.

In [4]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import numpy as np
import pandas as pd


num_samples = 500
np.random.seed(42)
data = pd.DataFrame({
    'Feature_A': np.random.normal(50, 15, num_samples),
    'Feature_B': np.random.uniform(10, 100, num_samples),
    'Feature_C': np.random.randint(1, 100, num_samples)
})


app = dash.Dash(__name__)


app.layout = html.Div([
    html.H2("Interactive Scatter Plot with User Controls"),
    
    html.Label("Select X-axis Feature:"),
    dcc.Dropdown(
        id='x-axis-dropdown',
        options=[{'label': col, 'value': col} for col in data.columns],
        value='Feature_A',
        clearable=False
    ),
    
    html.Label("Select Y-axis Feature:"),
    dcc.Dropdown(
        id='y-axis-dropdown',
        options=[{'label': col, 'value': col} for col in data.columns],
        value='Feature_B',
        clearable=False
    ),
    
    dcc.Graph(id='scatter-plot')
])


@app.callback(
    Output('scatter-plot', 'figure'),
    Input('x-axis-dropdown', 'value'),
    Input('y-axis-dropdown', 'value')
)
def update_scatter(x_feature, y_feature):
    fig = px.scatter(data, x=x_feature, y=y_feature, title=f'Scatter Plot: {x_feature} vs {y_feature}', 
                     labels={x_feature: x_feature, y_feature: y_feature})
    return fig

if __name__ == '__main__':
    app.run_server(debug=True  , port=8056)